# REAL OR FAKE NEWS ANALYSIS

ABOUT THE DATASET

This dataset consists of about 40000 articles consisting of fake as well as real news. Our aim is train our model so that it can correctly predict whether a given piece of news is real or fake.The fake and real news data is given in two separate datasets with each dataset consisting around 20000 articles each.

## Business Objective:

1.	Need to classify the fake and real news accurately.

Architecture level analysis:

1.	Data transformation/Text processing using Python
2.	Need to get sentiments Analysis and n-gram analysis with some charts like histogram, Density plot, Barplot, pie-plot etc. 
3.	Deployment through R Shiny or Flask/ Streamlit



In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import spacy
import seaborn as sns

In [5]:
spacy.__version__

'3.5.3'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
fake=pd.read_csv('Fake.csv',encoding='latin-1')

In [4]:
true=pd.read_csv('True.csv',encoding='latin-1',error_bad_lines=False)

Skipping line 11067: expected 4 fields, saw 8



In [5]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [6]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21416 entries, 0 to 21415
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21416 non-null  object
 1   text     21416 non-null  object
 2   subject  21416 non-null  object
 3   date     21416 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


#### so we can see that all the data in our dataset is catagorical
NOTE: You can see that all of the datetime related columns are not currently in datetime format. We will need to convert these later.

In [7]:
fake.shape

(23481, 4)

In [8]:
true.shape

(21416, 4)

fake dataset have 23481 ROWS and 4 COLUMNS

true dataset have 21416 ROWS and 4 COLUMNS

# Checking for Missing Data 

In [9]:
def missing_data(df):
    """
    Objective
    ----------
    it shows the missing data in each column with 
    total missing values, percentage of missing value and
    its data type in descending order.
    
    parameters
    ----------
    df: pandas dataframe
        input data frame 
    
    returns
    ----------
    missing_data: output data frame(pandas dataframe)
    
    """
    
    total = df.isna().sum().sort_values(ascending=False)
    percent = round((df.isnull().sum()/df.isna().count()  * 100).sort_values(ascending=False))
    data_type = df.dtypes
    missing_data = pd.concat([total,percent,data_type],axis=1,keys=['Total','Percent','Data_Type']).sort_values("Total", axis = 0, ascending = False)
    
    return missing_data

In [10]:
missing_data(true)

,Total,Percent,Data_Type
title,0,0.0,object
text,0,0.0,object
subject,0,0.0,object
date,0,0.0,object


In [11]:
missing_data(fake)

,Total,Percent,Data_Type
title,0,0.0,object
text,0,0.0,object
subject,0,0.0,object
date,0,0.0,object


we can clearly conclude that we don't have any missing data in our dataset .... GOOD to GO!!

# Lets Check For Duplicates in our Dataset 

In [12]:
def drop_duplicates(df):
    """
    Objective
    ----------
    Drop duplicates rows in data frame except for the first occurrence.
    
    parameters
    ----------
    df: pandas dataframe
        input data frame 
        
    returns
    ----------
    dataframe with all unique rows
    """
        
    try:
        dr = df.duplicated().value_counts()
        print("[INFO] Dropping {} duplicates records...".format(dr))
        f_df = df.drop_duplicates(keep="first")
        
        return f_df
    except KeyError:
        print("[INFO] No duplicates records found")
        return df

In [13]:
true=drop_duplicates(true)

[INFO] Dropping False    21210
True       206
dtype: int64 duplicates records...


In [14]:
fake=drop_duplicates(fake)

[INFO] Dropping False    23478
True         3
dtype: int64 duplicates records...


In [15]:
true.shape

(21210, 4)

In [16]:
fake.shape

(23478, 4)

### Statistics View
Now let's look at some statistics about the datasets

In [17]:
fake.describe()

,title,text,subject,date
count,23478,23478,23478,23478
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [18]:
true.describe()

,title,text,subject,date
count,21210,21210,21210,21210
unique,20825,21191,2,716
top,Factbox: Trump fills top jobs for his administ...,"CHARLESTON, S.C. (Reuters) - Long viewed by th...",politicsNews,"December 6, 2017"
freq,14,2,11219,166


# Feature_Engineering

In [19]:
true.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [20]:
fake.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [21]:
fake['subject'] = fake['subject'].replace(['News'], 'worldnews')
fake['subject'] = fake['subject'].replace(['left-news'], 'worldnews')
fake['subject'] = fake['subject'].replace(['US_News'], 'worldnews')
fake['subject'] = fake['subject'].replace(['Middle-east'], 'worldnews')
fake['subject'] = fake['subject'].replace(['politics'], 'politicsNews')
fake['subject'] = fake['subject'].replace(['Government News'], 'politicsNews')

In [22]:
fake.subject.unique()

array(['worldnews', 'politicsNews'], dtype=object)

# Converting the date feature into months and years

In [23]:
from datetime import datetime
import calendar
true['date'] = pd.to_datetime(true['date'], errors='coerce')
num = true['date'].dt.month

In [24]:
fake['date'] = pd.to_datetime(fake['date'], errors='coerce')
num = fake['date'].dt.month

In [25]:
true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21210 entries, 0 to 21415
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    21210 non-null  object        
 1   text     21210 non-null  object        
 2   subject  21210 non-null  object        
 3   date     21210 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 828.5+ KB


In [26]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23478 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    23478 non-null  object        
 1   text     23478 non-null  object        
 2   subject  23478 non-null  object        
 3   date     23468 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 917.1+ KB


In [27]:
fake['Month'] = fake.date.apply(lambda x:x.month)
fake['Year'] = fake.date.apply(lambda x:x.year)

In [28]:
true['Month'] = true.date.apply(lambda x:x.month)
true['Year'] = true.date.apply(lambda x:x.year)

In [29]:
fake.head()

,title,text,subject,date,Month,Year
0,Donald Trump Sends Out Embarrassing New Year...,Donald Trump just couldn t wish all Americans ...,worldnews,2017-12-31,12.0,2017.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,worldnews,2017-12-31,12.0,2017.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",worldnews,2017-12-30,12.0,2017.0
3,Trump Is So Obsessed He Even Has Obamas Name...,"On Christmas day, Donald Trump announced that ...",worldnews,2017-12-29,12.0,2017.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,worldnews,2017-12-25,12.0,2017.0


In [30]:
del fake['date']
del true['date']

In [31]:
true.head()

,title,text,subject,Month,Year
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,12,2017
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,12,2017
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,12,2017
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,12,2017
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,12,2017


In [32]:
fake.head()

,title,text,subject,Month,Year
0,Donald Trump Sends Out Embarrassing New Year...,Donald Trump just couldn t wish all Americans ...,worldnews,12.0,2017.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,worldnews,12.0,2017.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",worldnews,12.0,2017.0
3,Trump Is So Obsessed He Even Has Obamas Name...,"On Christmas day, Donald Trump announced that ...",worldnews,12.0,2017.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,worldnews,12.0,2017.0


# Data Cleaning
Now we are moving ahead with cleaning our data

Removal of HTML Contents

Removal of Punctuation Marks and Special Characters

Removal of Stopwords

Lemmatization

### First of all we are  mearging our tile,subject with text in to a single column 'text' to continue with our data cleaning process

In [33]:
fake['text']= fake['subject'] + " " + fake['title'] + " " + fake['text']
del fake['subject']
del fake['title']
fake.head()

,text,Month,Year
0,worldnews Donald Trump Sends Out Embarrassing...,12.0,2017.0
1,worldnews Drunk Bragging Trump Staffer Starte...,12.0,2017.0
2,worldnews Sheriff David Clarke Becomes An Int...,12.0,2017.0
3,worldnews Trump Is So Obsessed He Even Has Ob...,12.0,2017.0
4,worldnews Pope Francis Just Called Out Donald...,12.0,2017.0


In [34]:
true['text']= true['subject'] + " " + true['title'] + " " + true['text']
del true['subject']
del true['title']
true.head()


,text,Month,Year
0,"politicsNews As U.S. budget fight looms, Repub...",12,2017
1,politicsNews U.S. military to accept transgend...,12,2017
2,politicsNews Senior U.S. Republican senator: '...,12,2017
3,politicsNews FBI Russia probe helped by Austra...,12,2017
4,politicsNews Trump wants Postal Service to cha...,12,2017


In [35]:
from bs4 import BeautifulSoup
import re
import nltk
import string
from nltk.corpus import stopwords  

In [36]:
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)


In [37]:
def cleaning(text):
    text = remove_html(text)
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("[0-9" "]+"," ",text)
    text = re.sub('[‘’“”…]', '', text)
    return text

clean = lambda x: cleaning(x)

In [38]:
true.head()

,text,Month,Year
0,"politicsNews As U.S. budget fight looms, Repub...",12,2017
1,politicsNews U.S. military to accept transgend...,12,2017
2,politicsNews Senior U.S. Republican senator: '...,12,2017
3,politicsNews FBI Russia probe helped by Austra...,12,2017
4,politicsNews Trump wants Postal Service to cha...,12,2017


In [39]:
fake.head()

,text,Month,Year
0,worldnews Donald Trump Sends Out Embarrassing...,12.0,2017.0
1,worldnews Drunk Bragging Trump Staffer Starte...,12.0,2017.0
2,worldnews Sheriff David Clarke Becomes An Int...,12.0,2017.0
3,worldnews Trump Is So Obsessed He Even Has Ob...,12.0,2017.0
4,worldnews Pope Francis Just Called Out Donald...,12.0,2017.0


# FAKE DATASET

In [40]:
fake['text']=fake['text'].apply(cleaning)


In [41]:
#removing stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [42]:
new_stopwords = ['trump','say','president','people','go','make','state','would','one','us','get','obama','clinton','time','take','donald','tell','also','worldnews','new','white','come','call','right','want','use','house','campaign','report','see','vote','give','work','could','first','back','many','republican','attack','country','government','need','support','two','politicsnews','years','election','unite','party','last','group','include','may','political','republicans','former','national','bill','law','help','year','accord']

In [43]:
stop.extend(new_stopwords)

In [44]:
#Lets perfom lamatization into entire dataset
def lemmatize_words(text):
        lemma = nltk.WordNetLemmatizer()
        words = text.split()
        words = [lemma.lemmatize(word,pos='v') for word in words]
        return ' '.join(words)

In [45]:
fake['text'] = fake['text'].apply(lemmatize_words)

In [46]:
fake['text'] = fake['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [47]:
#word frequency after lamatization in fake dataset
freq_fk = pd.Series(' '.join(fake['text']).split()).value_counts()[:20] # for top 100
freq_fk

like        18756
know        16884
hillary     15219
news        14860
even        14436
image       14184
think       13284
show        11678
media       11620
via         11177
america     11059
video       11014
american    10240
ask          9073
police       9034
black        8955
way          8625
live         8574
try          8359
claim        8340
dtype: int64

In [48]:
fake['text']

0        send embarrass years eve message disturb wish...
1        drink brag staffer start russian collusion inv...
2        sheriff david clarke become internet joke thre...
3        obsess even obamas name cod website image chr...
4        pope francis christmas speech pope francis ann...
                               ...                        
23476    mcpain john mccain furious iran treat sailors ...
23477    justice yahoo settle email privacy classaction...
23478    sunnistan ally safe zone plan territorial bo...
23479    blow million al jazeera america finally quit c...
23480    navy sailors hold iranian military  sign neoc...
Name: text, Length: 23478, dtype: object

# TRUE DATASET

In [49]:
true['text']=true['text'].apply(cleaning)

In [50]:
true['text'] = true['text'].apply(lemmatize_words)

In [51]:
#removing stopwords
#stop = stopwords.words('english')
true['text'] = true['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [52]:
#word frequency after removal of stopwords in fake dataset
freq_tr = pd.Series(' '.join(true['text']).split()).value_counts()[:20] # for top 20
freq_tr

reuters         28071
washington      12172
trumps         11558
court           10616
north           10547
plan            10486
meet            10485
security        10267
percent          9942
tax              9868
minister         9857
senate           9652
rule             9464
force            8568
officials        8561
china            8392
foreign          8369
democratic       8343
since            8255
presidential     8222
dtype: int64

In [53]:
true['text']

0        budget fight loom flip fiscal script washingto...
1        military accept transgender recruit monday pen...
2        senior senator let mr mueller job washington r...
3        fbi russia probe australian diplomat tipoff ny...
4        postal service charge much amazon shipments se...
                               ...                        
21411    fully commit nato approach afghanistan brussel...
21412    lexisnexis withdraw products chinese market lo...
21413    minsk cultural hub become authorities minsk re...
21414    vatican upbeat possibility pope francis visit ...
21415    indonesia buy billion worth russian jet jakart...
Name: text, Length: 21210, dtype: object

# We can see that in true news we have more words in comparision to fake news

## Creating a dataframe 'data' 
##### Adding an Target vaeriable into our data frame assigning 1 to true and 0 to fake

In [54]:
#add column 
true['target'] = 1
fake['target'] = 0 

In [55]:
data=pd.concat([true,fake],ignore_index=True,sort=False)

In [56]:
from sklearn.utils import shuffle
data=shuffle(data)
data=data.reset_index(drop=True)


In [57]:
data.head()

,text,Month,Year,target
0,supporter rail hillarys place front fire squ...,9.0,2016.0,0
1,tim kaine show decent way respond shameful dwy...,8.0,2016.0,0
2,budget cut shut every public school kansas kan...,5.0,2016.0,0
3,norways rightwing win reelection fight oil tax...,9.0,2017.0,1
4,watch stephen colbert moderate hilarious vs de...,1.0,2016.0,0


# Data visualization

# Model Building 



In [58]:
data.head()

,text,Month,Year,target
0,supporter rail hillarys place front fire squ...,9.0,2016.0,0
1,tim kaine show decent way respond shameful dwy...,8.0,2016.0,0
2,budget cut shut every public school kansas kan...,5.0,2016.0,0
3,norways rightwing win reelection fight oil tax...,9.0,2017.0,1
4,watch stephen colbert moderate hilarious vs de...,1.0,2016.0,0


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [74]:
vectorizer = TfidfVectorizer(stop_words='english',max_features=3000, max_df =1.0, smooth_idf=True) #keep top 3000 words
#doc_vec = vectorizer.fit_transform(data["text"])
#names_features = vectorizer.get_feature_names_out()
x= vectorizer.fit_transform(data["text"]).toarray()

In [75]:
x

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.47322452]])

In [62]:
)

In [63]:
x

,abandon,abc,abe,abedin,ability,able,abortion,abroad,absolute,absolutely,...,young,younger,youth,youtube,zealand,zero,zika,zimbabwe,zone,zuma
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.059668,0.0,0.086817,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.069102,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.099993,0.0,0.098214,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44683,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
44684,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
44685,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
44686,0.0,0.0,0.0,0.0,0.0,0.030204,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [76]:
y=data['target']

In [77]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [79]:
rfc = RandomForestClassifier(n_estimators=50, random_state=2)

In [80]:
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(precision_score(y_test,y_pred))

0.996755426269859
[[4627   15]
 [  14 4282]]
0.9965091924598557


In [81]:
from sklearn.metrics import classification_report

In [82]:
print("Classification report:")
print(classification_report(y_test,y_pred))

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4642
           1       1.00      1.00      1.00      4296

    accuracy                           1.00      8938
   macro avg       1.00      1.00      1.00      8938
weighted avg       1.00      1.00      1.00      8938



In [85]:
import joblib
filename = "model.joblib"
joblib.dump(rfc,filename)

['model.joblib']

In [86]:
tfidf_obj = vectorizer.fit(data['text'])
joblib.dump(tfidf_obj, 'tf-idf.joblib')

['tf-idf.joblib']

In [91]:
tf_idf_converter = joblib.load("tf-idf.joblib")
classifier= joblib.load("model.joblib")

In [99]:
sent = ["WEST PALM BEACH, Fla (Reuters) - President Donald Trump said on Thursday he believes he will be fairly treated in a special counsel investigation into Russian meddling in the U.S. presidential election, but said he did not know how long the probe would last. The federal investigation has hung over Trumps White House since he took office almost a year ago, and some Trump allies have in recent weeks accused the team of Justice Department Special Counsel Robert Mueller of being biased against the Republican president. But in an interview with the New York Times, Trump appeared to shrug off concerns about the investigation, which was prompted by U.S. intelligence agencies conclusion that Russia tried to help Trump defeat Democrat Hillary Clinton by hacking and releasing embarrassing emails and disseminating propaganda. Theres been no collusion. But I think hes going to be fair, Trump said in what the Times described as a 30-minute impromptu interview at his golf club in West Palm Beach, Florida. Mueller has charged four Trump associates in his investigation. Russia has denied interfering in the U.S. election. U.S. Deputy Attorney General Rod Rosenstein said this month that he was not aware of any impropriety by Muellers team. Trumps lawyers have been saying for weeks that they had expected the Mueller investigation to wrap up quickly, possibly by the end of 2017. Mueller has not commented on how long it will last. Trump told the Times that he did not know how long the investigation would take. Timing-wise, I cant tell you. I just dont know, he said. Trump said he thought a prolonged probe makes the country look bad but said it has energized his core supporters. What its done is, its really angered the base and made the base stronger. My base is strong than its ever been, he said. The interview was a rare break in Trumps Christmas vacation in Florida. He has golfed each day aside from Christmas Day, and mainly kept a low profile, apart from the occasional flurry of tweets. He spent one day golfing with Republican Senator David Perdue from Georgia, who has pushed legislation to cap immigration numbers, and had dinner on Thursday with Commerce Secretary Wilbur Ross, an international trade hawk. Trump told the Times he hoped to work with Democrats in the U.S. Congress on a spending plan to fix roads and other infrastructure, and on protections for a group of undocumented immigrants who were brought to the United States as children. Trump spoke about trade issues, saying he had backed off his hard line on Chinese trade practices in the hope that Beijing would do more to pressure North Korea to end its nuclear and missile testing program. He said he had been disappointed in the results. He also complained about the North American Free Trade Agreement (NAFTA), which his administration is attempting to renegotiate in talks with Mexico and Canada. Trump said Canadian Prime Minister Justin Trudeau had played down the importance of Canadian oil and lumber exports to the United States when looking at the balance of trade between the two countries. If I dont make the right deal, Ill terminate NAFTA in two seconds. But were doing pretty good, Trump said."]

In [100]:
prediction = classifier.predict(tf_idf_converter.transform(sent))



In [101]:
print(prediction)
if (prediction[0]==0):
    print('The news is FAKE')
else:
    print('The news is True')

[1]
The news is True


In [ ]:
prediction=rfc.predict()
    
print(prediction)
if (prediction[0]==0):
    print('The news is FAKE')
else:
    print('The news is True')

In [102]:
import pickle
pickle_out=open("rfc.pkl","wb")
pickle.dump(rfc,pickle_out)
pickle_out.close()

In [103]:
pickle_out=open("classifier.pkl","wb")
pickle.dump(classifier,pickle_out)
pickle_out.close()

In [104]:
pickle_out=open("tf_idf_converter.pkl","wb")
pickle.dump(tf_idf_converter,pickle_out)
pickle_out.close()